In [1]:
ROBOT = "@"
WALL = "#"
BOX = "O"
FREE = "."
LEFT_BOX = "["
RIGHT_BOX = "]"

UP = (-1, 0)
DOWN = (1, 0)
RIGHT = (0, 1)
LEFT = (0, -1)

char_to_move = {
    "^": UP,
    ">": RIGHT,
    "v": DOWN,
    "<": LEFT,
}

def add(pos1, pos2):
    return ((pos1[0] + pos2[0]), (pos1[1] + pos2[1]))

def element_at_pos(pos, map):
    return map[pos[0]][pos[1]]

def get_robot_position(map):
    for i, row in enumerate(map):
        for j, element in enumerate(row):
            if element == ROBOT:
                return (i, j)

def parse_map(map):
    lines = map.split("\n")
    return [
        [x for x in row]
        for row in lines
    ]

def parse_moves(moves):
    moves = "".join(moves.split("\n"))
    return [
        char_to_move[move]
        for move in moves
    ]

def enlarge_map(map):
    new_map = []
    for row in map:
        new_row = []
        for element in row:
            if element == BOX:
                new_row.extend([LEFT_BOX, RIGHT_BOX])
            elif element == FREE:
                new_row.extend([FREE, FREE])
            elif element == WALL:
                new_row.extend([WALL, WALL])
            elif element == ROBOT:
                new_row.extend([ROBOT, FREE])
        new_map.append(new_row)
    return new_map

def sort_shifts(x, move):
    if move == UP:
        return x[0]
    elif move == DOWN:
        return -x[0]
    elif move == RIGHT:
        return -x[1]
    return x[1]

def shift_boxes(map, boxes_to_shift, move):
    boxes_to_shift = sorted(
        list(boxes_to_shift),
        key=lambda x: sort_shifts(x, move)
    )

    for current_pos in boxes_to_shift:
        row, col = current_pos
        new_row, new_col = add(current_pos, move)
        map[new_row][new_col] = element_at_pos(current_pos, map)
        map[row][col] = FREE
    return map

def apply_moves(map, moves):
    robot_pos = get_robot_position(map)
    for move in moves:
        boxes_to_shift = set()
        positions_to_check = set()
        positions_to_check.add(robot_pos)
        can_move = True
        while len(positions_to_check) > 0:
            position = positions_to_check.pop()
            boxes_to_shift.add(position)
            next_pos = add(position, move)
            element = element_at_pos(next_pos, map)
            while element != FREE:
                boxes_to_shift.add(next_pos)
                if element == WALL:
                    can_move = False
                    break
                if move in [UP, DOWN]:
                    if element == LEFT_BOX:
                        other = add(next_pos, RIGHT)
                        positions_to_check.add(other)
                    elif element == RIGHT_BOX:
                        other = add(next_pos, LEFT)
                        positions_to_check.add(other)
                next_pos = add(next_pos, move)
                element = element_at_pos(next_pos, map)
            if not can_move:
                break
        if can_move:
            robot_pos = add(robot_pos, move)
            map = shift_boxes(map, boxes_to_shift, move)
    return map

def calculate_score(map):
    score = 0
    for i, row in enumerate(map):
        for j, element in enumerate(row):
            if element in [BOX, LEFT_BOX]:
                score += 100 * i + j
    return score

In [2]:
input_file = "data/input.txt"

with open(input_file, 'r') as f:
    map, moves = f.read().split("\n\n")
    map = parse_map(map)
    moves = parse_moves(moves)
    large_map = enlarge_map(map)

    map = apply_moves(map, moves)
    large_map = apply_moves(large_map, moves)

    score = calculate_score(map)
    score2 = calculate_score(large_map)
    print(f"Answer 1: {score}")
    print(f"Answer 2: {score2}")

Answer 1: 1398947
Answer 2: 1397393
